In [1]:
# Load Packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import os
import time
import gc

In [2]:
# Load data
dat_train = pd.read_csv(os.getcwd()+'\\sales_dir\\train.csv')
dat_stores = pd.read_csv(os.getcwd()+'\\sales_dir\\stores.csv')
dat_features = pd.read_csv(os.getcwd()+'\\sales_dir\\features.csv')

In [3]:
# Understand dataset structure

dataset_names = ['dat_train', 'dat_stores', 'dat_features']
datasets = [dat_train, dat_stores, dat_features]
for dataset in datasets:
    print(dataset.head())
    print('---------')
    print(dataset.columns)
    print(dataset.dtypes)
    print('--------------')

   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
---------
Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday'], dtype='object')
Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
dtype: object
--------------
   Store Type    Size
0      1    A  151315
1      2    A  202307
2      3    B   37392
3      4    A  205863
4      5    B   34875
---------
Index(['Store', 'Type', 'Size'], dtype='object')
Store     int64
Type     object
Size      int64
dtype: object
--------------
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2

In [4]:
# Since we will be merging on date column as well, first lets convert those

dat_train['Date'] = pd.to_datetime(dat_train['Date'])
dat_features['Date'] = pd.to_datetime(dat_features['Date'])

In [5]:
# Merge datasets

dat_train_f = dat_train.merge(dat_features,how = 'left', on = ['Store', 'Date', 'IsHoliday'])
dat_train_fs = dat_train_f.merge(dat_stores,how = 'left', on = ['Store'])

In [6]:
# Remove the unwanted datasets to freeup space in notebook
dat_train = None
dat_train_f = None
dat_features = None
dat_stores = None

In [7]:
# IsHoliday # Convert boolean to int

dat_train_fs = dat_train_fs.applymap(lambda x:0 if x==False else x)
dat_train_fs = dat_train_fs.applymap(lambda x:1 if x==True else x)

In [8]:
# Type # Dummy code categorical variable

dat_train_fs = pd.get_dummies(dat_train_fs, columns = ['Type'])

In [9]:
# check for missing data
def check_nan_inf(df):
    checks_df = pd.DataFrame(df.isnull().sum()).T
    checks_df = checks_df.append(pd.DataFrame(df.isin([np.inf,-np.inf]).sum()).T)
    checks_df = checks_df.append(pd.DataFrame(df.isnull().sum()/df.shape[0]).T)
    checks_df['names'] = ['nulls','infs','%nulls']
    checks_df = checks_df.set_index('names')
    return checks_df

In [10]:
%time
checks_train = check_nan_inf(dat_train_fs)

Wall time: 0 ns


In [11]:
checks_train

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,Type_A,Type_B,Type_C
names,,,,,,,,,,,,,,,,,,
nulls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,270889.000000,310322.00000,284479.000000,286603.000000,270138.00000,0.0,0.0,0.0,0.0,0.0,0.0
infs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
%nulls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.642572,0.73611,0.674808,0.679847,0.64079,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Replace all nan's with 0

dat_train_fs.fillna(0, inplace = True)

In [13]:
# Since we are looking at markdowns as a single component that affects demand, we will add it as one column.
dat_train_fs['Markdown_total'] = dat_train_fs[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].sum(axis = 1)

In [14]:
dat_train_fs['year'] = dat_train_fs['Date'].dt.year
dat_train_fs = dat_train_fs.loc[dat_train_fs['year']==2012]
dat_train_fs = dat_train_fs.reset_index(drop = True)

In [15]:
dat_train_fs['month'] = dat_train_fs['Date'].dt.month

In [16]:
dat_train_fs.shape

(127438, 21)

In [17]:
dat_train_fs.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Size', 'Type_A', 'Type_B',
       'Type_C', 'Markdown_total', 'year', 'month'],
      dtype='object')

In [18]:
# remove unwanted columns
dat_train_fs = dat_train_fs.drop(columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5','year'])

In [19]:
dat_train_fs.shape

(127438, 15)

In [20]:
dat_train_fs.to_csv('sales_dir\sales_data.csv', index = False)